# Exploracion del Dataset: Old Book Illustrations

Analizamos el dataset `gigant/oldbookillustrations` para entender su estructura
y definir las transformaciones necesarias para el fine-tuning de Stable Diffusion.

In [ ]:
from datasets import load_dataset
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

## 1. Cargar dataset y ver columnas disponibles

In [ ]:
dataset_name = "gigant/oldbookillustrations"
dataset = load_dataset(dataset_name, split="train")

print(f"Numero de muestras: {len(dataset)}")
print(f"Columnas: {dataset.column_names}")
print(f"\nEjemplo de una fila:")
print(dataset[0])

## 2. Analizar tamanos de imagenes

**Punto clave:** A diferencia del dataset Pokemon del docente (imagenes cuadradas),
las ilustraciones de libros antiguos tienen proporciones variadas. Esto afecta
directamente a como definimos las transformaciones.

In [ ]:
# Recopilar tamanos de todas las imagenes
widths = []
heights = []
aspect_ratios = []

for i in range(len(dataset)):
    img = dataset[i]["1600px"]
    w, h = img.size
    widths.append(w)
    heights.append(h)
    aspect_ratios.append(w / h)

print(f"Ancho  - min: {min(widths)}, max: {max(widths)}, media: {np.mean(widths):.0f}")
print(f"Alto   - min: {min(heights)}, max: {max(heights)}, media: {np.mean(heights):.0f}")
print(f"Aspect ratio - min: {min(aspect_ratios):.2f}, max: {max(aspect_ratios):.2f}, media: {np.mean(aspect_ratios):.2f}")

# Cuantas son cuadradas?
cuadradas = sum(1 for w, h in zip(widths, heights) if w == h)
print(f"\nImagenes cuadradas: {cuadradas}/{len(dataset)} ({100*cuadradas/len(dataset):.1f}%)")

In [ ]:
# Histogramas de tamanos
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(widths, bins=30, color='steelblue', edgecolor='black')
axes[0].set_title('Distribucion de anchos')
axes[0].set_xlabel('Ancho (px)')

axes[1].hist(heights, bins=30, color='coral', edgecolor='black')
axes[1].set_title('Distribucion de altos')
axes[1].set_xlabel('Alto (px)')

axes[2].hist(aspect_ratios, bins=30, color='mediumseagreen', edgecolor='black')
axes[2].set_title('Distribucion de aspect ratios')
axes[2].set_xlabel('Aspect Ratio (w/h)')
axes[2].axvline(x=1.0, color='red', linestyle='--', label='Cuadrada')
axes[2].legend()

plt.tight_layout()
plt.show()

## 3. Visualizar muestras con sus captions

In [ ]:
# Mostrar 8 muestras del dataset
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
indices = np.random.choice(len(dataset), 8, replace=False)

for ax, idx in zip(axes.flat, indices):
    example = dataset[int(idx)]
    img = example["1600px"]
    caption = example["info_alt"]
    ax.imshow(img)
    ax.set_title(caption[:60] + "..." if len(caption) > 60 else caption, fontsize=9)
    ax.axis('off')

plt.suptitle('Muestras del dataset Old Book Illustrations', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Comparar transformaciones

Comparamos dos estrategias de resize:
- **Resize(512, 512):** Estira la imagen. Es lo que usa el docente con Pokemon (imagenes ya cuadradas).
- **Resize(512) + CenterCrop(512):** Primero redimensiona el lado menor a 512, luego recorta el centro. Preserva proporciones.

Para imagenes no cuadradas como las de este dataset, la segunda opcion es mejor.

In [ ]:
# Definir las dos transformaciones
transform_stretch = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

transform_crop = transforms.Compose([
    transforms.Resize(512),
    transforms.CenterCrop(512),
    transforms.ToTensor(),
])

# Comparar con 4 imagenes del dataset
fig, axes = plt.subplots(4, 3, figsize=(15, 20))

sample_indices = [0, 10, 50, 100]
for row, idx in enumerate(sample_indices):
    img = dataset[idx]["1600px"].convert("RGB")
    w, h = img.size
    
    stretched = transforms.ToPILImage()(transform_stretch(img))
    cropped = transforms.ToPILImage()(transform_crop(img))
    
    axes[row, 0].imshow(img)
    axes[row, 0].set_title(f'Original ({w}x{h})', fontsize=10)
    axes[row, 0].axis('off')
    
    axes[row, 1].imshow(stretched)
    axes[row, 1].set_title('Resize(512,512) - Estira', fontsize=10)
    axes[row, 1].axis('off')
    
    axes[row, 2].imshow(cropped)
    axes[row, 2].set_title('Resize(512) + CenterCrop(512)', fontsize=10)
    axes[row, 2].axis('off')

plt.suptitle('Comparacion de transformaciones', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Verificar integridad del dataset

Comprobamos que no hay imagenes rotas ni captions vacias.

In [ ]:
# Verificar integridad
resolution = 512
image_transforms = transforms.Compose([
    transforms.Resize(resolution),
    transforms.CenterCrop(resolution),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

errores = 0
captions_vacios = 0

for i in range(len(dataset)):
    example = dataset[i]
    
    # Verificar caption
    caption = example["info_alt"]
    if not caption or caption.strip() == "":
        captions_vacios += 1
    
    # Verificar imagen
    try:
        img = example["1600px"].convert("RGB")
        tensor = image_transforms(img)
        assert tensor.shape == (3, 512, 512)
    except Exception as e:
        print(f"Error en muestra {i}: {e}")
        errores += 1

print(f"\nResultados de verificacion:")
print(f"  Total muestras: {len(dataset)}")
print(f"  Imagenes con error: {errores}")
print(f"  Captions vacios: {captions_vacios}")
print(f"  Todas las imagenes se procesan correctamente: {errores == 0}")

## Conclusiones

1. Las imagenes del dataset **no son cuadradas** (a diferencia del dataset Pokemon del docente).
2. Usaremos `Resize(512) + CenterCrop(512)` en vez de `Resize((512, 512))` para no distorsionar.
3. La columna de imagen es `"1600px"` (en vez de `"image"` del Pokemon).
4. La columna de caption es `"info_alt"` (en vez de `"text"` del Pokemon).
5. Todas las muestras se procesan correctamente con la transformacion elegida.